# OpenWeather DataFrame for ETL Project

We get our Auto Insurance Data Add on our weather data via our API call 

#### Import dependencies

In [268]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
from pprint import pprint
import os
import pandas as pd
import csv

In [269]:
# Import Auto Insurance data
auto_data_path = os.path.join("Resources", "Temp_Average_Auto_Insurance_Data.csv")
auto_data_df = pd.read_csv(auto_data_path)

In [270]:
auto_data_df

,State,Zip_Code,City,Average_Auto_Insurance_Rate
0,Massachusetts,2119,Roxbury,"$2,943"
1,Massachusetts,2119,Boston,"$5,333.79"
2,Massachusetts,2554,Nantucket,"$2,378.90"
3,Rhode Island,2842,Middletown,"$3,908.01"
4,Rhode Island,2907,Providence,"$7,262.39"
...,...,...,...,...
220,Washington,98118,Seattle,"$4,079.23"
221,Washington,98368,Port Townsend,"$2,461.54"
222,Alaska,99504,Anchorage,"$1,877"
223,Alaska,99504,Anchorage,"$3,132.74"


In [271]:
auto_data_list = auto_data_df.values.tolist()
zip_list = auto_data_df.Zip_Code.tolist() 
city_list = auto_data_df.City.tolist()

In [272]:
# zip_list_test = zip_list[0:5]
# zip_list_test

#### Save configuration and build query URL

In [273]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial city query URL
query_url = f"{url}appid={api_key}&units={units}&q="
print(query_url)

# Build partial zip code query URL
query2_url = f"{url}&units={units}&q="
print(query2_url)

http://api.openweathermap.org/data/2.5/weather?appid=7e98628ecd4395a6ca3f342c5abe7ccc&units=metric&q=
http://api.openweathermap.org/data/2.5/weather?&units=metric&q=


#### Loop through list of cities to request information for each

In [274]:
# # Loop with a list of zipcodes
# # zip_test = ["77077", "77407", "77494"]

# # set up lists to hold reponse info
# lat_list = []
# lon_list = []
# hum_list = []
# temp_list = []
# maxTemp_list = []
# minTemp_list = []
# cloud_list = []
# wind_list = []
# city_list = []
# weather_list = []
# description_list =[]

# # Loop through the list of cities and perform a request for data on each
# for city in city_list:
#     weather_url = query_url + city
# #     print(weather_url)
#     response = requests.get(weather_url).json()
# #     pprint(response)
#     try:
#         lon_list.append(response['coord']['lon'])
#         lat_list.append(response['coord']['lat'])
#     except:
#         print(f"Oops, looks like we didn't get something (lat or long) from this zip code: {zips} gave us a problem. On to the next one")
#         lon_list.append('NaN')
#         lat_list.append('NaN')
#     try:
#         weather_list.append(response['weather'][0]['main'])
#         description_list.append(response['weather'][0]['description'])
#     except:
#         print(f"Oops, looks like we didn't get something (weather or description) from this zip code: {zips} gave us a problem. On to the next one")
#         weather_list.append('NaN')
#         description_list.append('NaN')
#     try:
#         temp_list.append(response['main']['temp'])
#         maxTemp_list.append(response['main']['temp_max'])
#         minTemp_list.append(response['main']['temp_min'])
#     except:
#         print(f"Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: {zips} gave us a problem. On to the next one")
#         temp_list.append('NaN')
#         maxTemp_list.append('NaN')
#         minTemp_list.append('NaN')
#     try:
#         hum_list.append(response['main']['humidity'])
#         cloud_list.append(response['clouds']['all'])
#         wind_list.append(response['wind']['speed'])
#         city_list.append(response['name'])
#     except:
#         print(f"Oops, looks like we didn't get something from this zip code: {zips} gave us a problem. On to the next one")
#         hum_list.append('NaN')
#         cloud_list.append('NaN')
#         wind_list.append('NaN')
#         city_list.append('NaN')

# # Create lat lon pairs
# latlongList = list(zip(lat_list, lon_list))
# # latlongList

In [275]:
# Loop with a list of zipcodes
# zip_test = ["77077", "77407", "77494"]

# set up lists to hold reponse info
lat_list = []
lon_list = []
hum_list = []
temp_list = []
maxTemp_list = []
minTemp_list = []
cloud_list = []
wind_list = []
city_list = []
weather_list = []
description_list =[]
# Loop through the list of cities and perform a request for data on each
for zips in zip_list:
    weather_url = f"{query2_url}{zips},US&appid={api_key}"
#     print(weather_url)
    response = requests.get(weather_url).json()
#     pprint(response)
    try:
        lon_list.append(response['coord']['lon'])
        lat_list.append(response['coord']['lat'])
    except:
        print(f"Oops, looks like we didn't get something (lat or long) from this zip code: {zips} gave us a problem. On to the next one")
        lon_list.append('NaN')
        lat_list.append('NaN')
    try:
        weather_list.append(response['weather'][0]['main'])
        description_list.append(response['weather'][0]['description'])
    except:
        print(f"Oops, looks like we didn't get something (weather or description) from this zip code: {zips} gave us a problem. On to the next one")
        weather_list.append('NaN')
        description_list.append('NaN')
    try:
        temp_list.append(response['main']['temp'])
        maxTemp_list.append(response['main']['temp_max'])
        minTemp_list.append(response['main']['temp_min'])
    except:
        print(f"Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: {zips} gave us a problem. On to the next one")
        temp_list.append('NaN')
        maxTemp_list.append('NaN')
        minTemp_list.append('NaN')
    try:
        hum_list.append(response['main']['humidity'])
        cloud_list.append(response['clouds']['all'])
        wind_list.append(response['wind']['speed'])
        city_list.append(response['name'])
    except:
        print(f"Oops, looks like we didn't get something from this zip code: {zips} gave us a problem. On to the next one")
        hum_list.append('NaN')
        cloud_list.append('NaN')
        wind_list.append('NaN')
        city_list.append('NaN')

# Create lat lon pairs
latlongList = list(zip(lat_list, lon_list))
# latlongList

Oops, looks like we didn't get something (lat or long) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something (weather or description) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something (lat or long) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something (weather or description) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get something from this zip code: 2119 gave us a problem. On to the next one
Oops, looks like we didn't get somet

#### Create a DataFrame from cities, lat, and temp

In [276]:
# now_list = maxTemp_list
# print(len(now_list))
# print(now_list)

weather_dict = {"Zip_Code": zip_list, "City": city_list, "Weather_Forecast": weather_list, "Weather_Description": description_list, "Max_Temperature": maxTemp_list, "Min_Temperature": minTemp_list, "Humidity": hum_list}
weather_df = pd.DataFrame(weather_dict)
weather_df

,Zip_Code,City,Weather_Forecast,Weather_Description,Max_Temperature,Min_Temperature,Humidity
0,2119,NaN,NaN,NaN,NaN,NaN,NaN
1,2119,NaN,NaN,NaN,NaN,NaN,NaN
2,2554,NaN,NaN,NaN,NaN,NaN,NaN
3,2842,NaN,NaN,NaN,NaN,NaN,NaN
4,2907,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
220,98118,Seattle,Clouds,scattered clouds,9.44,6.67,34
221,98368,Everett,Clouds,broken clouds,8.89,6.11,61
222,99504,Anchorage,Rain,light rain,2.22,0.56,80
223,99504,Anchorage,Rain,light rain,2.22,0.56,80


In [278]:
# Combine the data into a single dataset
Auto_data_weather_v0_df = pd.merge(auto_data_df, weather_df, how='left', on='Zip_Code')
Auto_data_weather_v1_df = Auto_data_weather_v0_df.rename(columns={'City_x':'City'})
Auto_data_weather_v1_df

# Output dataset
output_file = os.path.join("Resources", "Average_Auto_Insurance_Weather_Data_Temp.csv")
Auto_data_weather_v1_df.to_csv(output_file, index = False)

#### Build a 7 day forecast table

In [ ]:
# # Create lat lon pairs
# latlongcityList = list(zip(lat_list, lon_list, city_list))
# # latlongcityList

# # for latlons in latlongList:
# #     print(f"lat = {latlons[0]}")
# #     print(f"lon = {latlons[1]}")

In [ ]:
# # Build 7 day URL
# # Save config information.
# url = "https://api.openweathermap.org/data/2.5/onecall?"
# units = "imperial"
# exclude = "hourly,minutely"

# # Build partial code query URL
# sevenday_query_url = f"{url}&units={units}&exclude={exclude}"
# # print(sevenday_query_url)

In [ ]:
# # # Loop with a list of zipcodes
# # zip_test = ["77077", "77407", "77494"]

# # # set up lists to hold reponse info
# # latlong_list = []
# # lon_list = []

# # # Loop through the list of cities and perform a request for data on each
# # for zips in zip_test:
# #     weather_url = f"{query_url}{zips},US&appid={api_key}"
# #     response = requests.get(weather_url).json()
# #     lat_list.append(response['coord']['lat'])
# #     lon_list.append(response['coord']['lon'])

# # # Make Latlong tuples
# latlongList = list(zip(lat_list, lon_list))
# days = [0, 1, 2]
# current_temp_list = []
# current_humidity_list = []
# current_wind_list = []
# Forecast_List = []
# Description_List = []
# Max_Temp_List = []
# Humidity_List = []
# rain_List = []

# # Loop through the list of tupules and perform a request for data on each
# for latlon in latlongList:
#     for day in days:
#         weather_url = f"{url}lat={latlon[0]}&lon={latlon[1]}&exclude={exclude}&units={units}&appid={api_key}"
#         response = requests.get(weather_url).json()
# #         pprint(response)
#         try:
#             current_temp_list.append(response['current']['temp'])
#             current_humidity_list.append(response['current']['humidity'])
#             current_wind_list.append(response['current']['wind'])
#             Max_Temp_List.append(response['daily'][day]['temp']['max'])
#             Humidity_List.append(response['daily'][day]['humidity'])
#             Forecast_List.append(response['daily'][day]['weather'][0]['main'])
#             Description_List.append(response['daily'][day]['weather'][0]['description'])
#             rain_List.append(response['daily'][day]['rain'])
#         except:
# #             print(f"Oops, looks like I didn't find some values on day {day}. On to the next one")
#             continue

In [ ]:
# now_list = Description_List
# print(len(now_list))
# print(now_list)

In [ ]:
# Forcast_dict = {"Zip_code": zip_test, "City": city_list, "Current Temperature": current_temp_list, "Current Humidity": current_humidity_list, "Current Wind speed": current_wind_list, "Day1 Forecast": Forecast_List, "Wind speed": wind_list, "Cloudiness": cloud_list}
# Forcast_df = pd.DataFrame(Forcast_dict)
# Forcast_df

In [ ]:
# current_temp_Dict = {}
# Temp7dayList = []
# current_humidity_Dict = {}
# current_wind_Dict = {}
# Max_Temp_Dict_7day = {}
# Humidity_Dict_7day = {}
# Forecast_Dict_7day = {}
# Description_Dict_7day = {}
# Rain_Dict_7day = {}

# # Loop through the list of tupules and perform a request for data on each
# for latlon in latlongList:
#     current_temp_Dict[latlon] = response['current']['temp']
#     Temp7dayList.append(response['current']['temp'])
#     current_humidity_Dict[latlon] = response['current']['humidity']
# #     current_wind_Dict = response['current']['wind']
#     for day in days:
#         weather_url = f"{url}lat={latlon[0]}&lon={latlon[1]}&exclude={exclude}&units={units}&appid={api_key}"
#         response = requests.get(weather_url).json()
#         try:
#             Max_Temp_Dict_7day[latlon][day] = response['daily'][day]['temp']['max']
#             Humidity_Dict_7day[latlon][day] = response['daily'][day]['humidity']
#             Forecast_Dict_7day[latlon][day] = response['daily'][day]['weather'][0]['main']
#             Description_Dict_7day[latlon][day] = response['daily'][day]['weather'][0]['description']
#         except:
# #             print(f"Oops, looks like this might be a problem on day {day}. On to the next one")
#             continue
#         try:
#             Rain_Dict_7day[latlon][day] = response['daily'][day]['rain']
#         except:
# #             print(f"Oops, looks like I didn't find some values for rain on day {day}. On to the next one")
#             continue

In [ ]:

# LatLongDictList = (current_temp_Dict.keys())
# LatLongDictList

# # # using list comprehension to 
# # # perform Unzipping 
# res = [[ i for i, j in LatLongDictList], 
#        [ j for i, j in LatLongDictList]] 
# LatList_7day = res[0]
# LonList_7day = res[1]

# LatLonTempList_7day = list(zip(LatList_7day, LonList_7day, Temp7dayList))
# # LatLonTempList_7day


# # Description_Dict_7day


In [ ]:
# cities7day = []

# for x, y, z  in latlongcityList:
#     for a, b, c in LatLonTempList_7day:
#         if (x==a) & (y==b):
#             cities7day.append(z)
            
# print(len(cities7day))
# print(cities7day)